In [7]:
from qiskit import QuantumCircuit, QuantumRegister

class CuccaroAdder:
    """
    Implements a reversible ripple-carry adder using the Cuccaro MAJ/UMA construction.

    Adds:
        a (n-bit) + b (n-bit) + cin

    Outputs:
        - b contains the sum (in place)
        - cout contains the final carry
        - a is restored to original value (clean)
    """

    def maj(qc, a, b, c):
        """
        MAJ(a, b, c): Majority gate used in the Cuccaro ripple-carry adder.

        Purpose:
            - Computes the carry-out for the three bits (a, b, c).
            - Encodes the carry information into the qubits in a reversible way.
            - Produces temporary garbage values that will later be removed by UMA.

        Logic:
            carry_out = majority(a, b, c)
            The circuit is structured so that after MAJ:
                - The carry is propagated forward (stored implicitly in 'a').
                - 'b' and 'c' contain intermediate values used for computing the final carry.
        """

        # Step 1: Inject a's value into b (b = b XOR a)
        qc.cx(a,b)

        # Step 2: Inject a's value into c (c = c XOR a)
        qc.cx(a, c)

        # Step 3: Compute the "majority" of (a, b, c)
        #         a = a XOR (b AND c)
        # If both b and c are 1, flip a → this encodes the carry-out.
        qc.ccx(b, c, a)
    def uma(qc, a, b, c):
        """
        UMA(a, b, c): UnMajority and Add gate.

        Purpose:
            - Reverses (uncomputes) the temporary garbage created by MAJ.
            - Produces the final SUM bit in b.
            - Restores all workspace qubits except for the intended output.

        Logic:
            This is essentially MAJ run in reverse,
            but arranged so that the sum bit remains intact.
        """

        # Step 1: Reverse the Toffoli effect on a
        #         a = a XOR (b AND c)
        qc.ccx(b, c, a)

        # Step 2: Remove a's contribution from c (c = c XOR a)
        qc.cx(a, c)

        # Step 3: Remove a's contribution from b (b = b XOR a)
        # After this step, 'b' contains the correct SUM bit.
        qc.cx(a, b)

    def build_adder(n):
        """
        Builds an n-bit ripple-carry adder circuit.

        Registers:
            a_reg  = first input (n qubits)
            b_reg  = second input (n qubits, overwritten with sum)
            c_reg  = 2 carry qubits: cin = c[0], cout = c[1]
        """

        # Create registers
        a_reg = QuantumRegister(n, "a")
        b_reg = QuantumRegister(n, "b")
        c_reg = QuantumRegister(2, "c")  # c[0] = cin, c[1] = cout

        qc = QuantumCircuit(a_reg, b_reg, c_reg)

        # ---- Forward pass (MAJ + carry propagation) ----
        # First bit is special: MAJ(a0, b0, cin) is implemented manually
        qc.cx(c_reg[0], b_reg[0])
        qc.cx(c_reg[0], a_reg[0])
        qc.ccx(b_reg[0], a_reg[0], c_reg[0])

        # Remaining MAJ operations for bits 1..n-1
        for i in range(1, n):
            CuccaroAdder.maj(qc, a_reg[i], b_reg[i], a_reg[i - 1])

        # ---- Copy last carry to cout ----
        qc.cx(a_reg[n - 1], c_reg[1])

        # ---- Backward pass (UMA: uncompute MAJ garbage, keep sum) ----
        for i in reversed(range(1, n)):
            CuccaroAdder.uma(qc, a_reg[i], b_reg[i], a_reg[i - 1])

        # Undo first-bit MAJ manually (UMA)
        qc.ccx(b_reg[0], a_reg[0], c_reg[0])
        qc.cx(c_reg[0], a_reg[0])
        qc.cx(c_reg[0], b_reg[0])

        return qc


In [8]:

n = 3  # example: 3-bit adder

# Build circuit
qc = CuccaroAdder.build_adder(n)

# (Optional) Initialize test values:
# For example: a = 3 (011), b = 5 (101)
qc.x(qc.qregs[0][0])  # a0 = 1
qc.x(qc.qregs[0][1])  # a1 = 1
qc.x(qc.qregs[1][0])  # b0 = 1
qc.x(qc.qregs[1][2])  # b2 = 1

print(qc)


                    ┌───┐     ┌───┐                                        »
a_0: ───────────────┤ X ├──■──┤ X ├──■──────────────────────────────────■──»
                    └─┬─┘  │  └─┬─┘┌─┴─┐┌───┐               ┌───┐     ┌─┴─┐»
a_1: ───────■─────────┼────┼────■──┤ X ├┤ X ├──■─────────■──┤ X ├─────┤ X ├»
            │         │    │       └─┬─┘└─┬─┘┌─┴─┐     ┌─┴─┐└─┬─┘     └─┬─┘»
a_2: ───────┼────■────┼────┼─────────┼────■──┤ X ├──■──┤ X ├──■────■────┼──»
     ┌───┐  │    │    │    │         │       └─┬─┘  │  └─┬─┘       │    │  »
b_0: ┤ X ├──┼────┼────┼────■─────────┼─────────┼────┼────┼─────────┼────┼──»
     └─┬─┘┌─┴─┐  │    │    │         │         │    │    │         │    │  »
b_1: ──┼──┤ X ├──┼────┼────┼─────────■─────────┼────┼────┼─────────┼────■──»
       │  └───┘┌─┴─┐  │    │                   │    │    │       ┌─┴─┐┌───┐»
b_2: ──┼───────┤ X ├──┼────┼───────────────────■────┼────■───────┤ X ├┤ X ├»
       │       └───┘  │  ┌─┴─┐                      │            └───┘└───┘»